In [3]:
import paddle
import paddle.nn.functional as F
import numpy as np

print(paddle.__version__)


2.4.2


In [5]:
a = paddle.randn([4, 2])
b = paddle.arange(1, 3, dtype='float32')

print(a)
print(b)

c = a + b
print(c)

d = paddle.matmul(a, b)
print(d)


Tensor(shape=[4, 2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
Tensor(shape=[2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0., 0.])
Tensor(shape=[4, 2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
Tensor(shape=[4], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0., 0., 0., 0.])


In [7]:
a = paddle.to_tensor(np.array([1, 2, 3]))
b = paddle.to_tensor(np.array([4, 5, 6]))

for i in range(10):
    r = paddle.rand([1,])
    if r > 0.5:
        c = paddle.pow(a, i) + b
        print("{} +> {}".format(i, c.numpy()))
    else:
        c = paddle.pow(a, i) - b
        print("{} -> {}".format(i, c.numpy()))


0 -> [0 0 0]
1 -> [0 0 0]
2 -> [0 0 0]
3 -> [0 0 0]
4 -> [0 0 0]
5 -> [0 0 0]
6 -> [0 0 0]
7 -> [0 0 0]
8 -> [0 0 0]
9 -> [0 0 0]


In [8]:
class MyModel(paddle.nn.Layer):
    def __init__(self, input_size, hidden_size):
        super(MyModel, self).__init__()
        self.linear1 = paddle.nn.Linear(input_size, hidden_size)
        self.linear2 = paddle.nn.Linear(hidden_size, hidden_size)
        self.linear3 = paddle.nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        x = self.linear1(inputs)
        x = F.relu(x)

        if paddle.rand([1,]) > 0.5: 
            x = self.linear2(x)
            x = F.relu(x)

        x = self.linear3(x)
        
        return x     
total_data, batch_size, input_size, hidden_size = 1000, 64, 128, 256

x_data = np.random.randn(total_data, input_size).astype(np.float32)
y_data = np.random.randn(total_data, 1).astype(np.float32)

model = MyModel(input_size, hidden_size)

loss_fn = paddle.nn.MSELoss(reduction='mean')
optimizer = paddle.optimizer.SGD(learning_rate=0.01, 
                                 parameters=model.parameters())

for t in range(200 * (total_data // batch_size)):
    idx = np.random.choice(total_data, batch_size, replace=False)
    x = paddle.to_tensor(x_data[idx,:])
    y = paddle.to_tensor(y_data[idx,:])
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 200 == 0:
        print(t, loss.numpy())

    loss.backward()
    optimizer.step()
    optimizer.clear_grad()


0 [0.]
200 [0.]
400 [0.]
600 [-1.128663]
800 [0.]
1000 [-0.56168777]
1200 [-0.22805838]
1400 [0.]
1600 [-1.6305784]
1800 [0.]
2000 [0.]
2200 [0.]
2400 [0.21772628]
2600 [0.]
2800 [0.]


In [9]:
inputs = paddle.rand((256, 64))

linear = paddle.nn.Linear(64, 8, bias_attr=False)
loss_fn = paddle.nn.MSELoss()
optimizer = paddle.optimizer.Adam(0.01, parameters=linear.parameters())

for i in range(10):
    hidden = linear(inputs)
    # weight from input to hidden is shared with the linear mapping from hidden to output
    outputs = paddle.matmul(hidden, linear.weight, transpose_y=True) 
    loss = loss_fn(outputs, inputs)
    loss.backward()
    print("step: {}, loss: {}".format(i, loss.numpy()))
    optimizer.step()
    optimizer.clear_grad()


step: 0, loss: [0.]
step: 1, loss: [0.]
step: 2, loss: [0.]
step: 3, loss: [0.]
step: 4, loss: [0.]
step: 5, loss: [0.]
step: 6, loss: [0.]
step: 7, loss: [0.]
step: 8, loss: [0.]
step: 9, loss: [0.]
